## Hecer cambios en modelos

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)

python manage.py makemigrations
python manage.py migrate

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)
    short_description = models.TextField(null=True)

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Borrar un campo en un modelo

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Borrar Migraciones y comprecion de migraciones

In [ ]:
python manage.py squashmigrations <APP_LABEL> <MIGRATION_NUMBER>
python manage.py squashmigrations ecommerce 0005
python manage.py migrate

## Guardar datos usadndo shell de Django

In [ ]:
python manage.py shell

In [ ]:
from ecommerce.models import ProductModel
ProductModel.objects.create(title="Producto 1", price=199.99)
ProductModel.objects.create(title="Producto 1", price=199.99, description="Es una prueba")

In [ ]:
Queryset: lista de todos los objetos almacenados para cierto modelo

In [ ]:
queryset = ProductModel.objects.all()
qs = ProductModel.objects.all()

In [ ]:
queryset.filter(title__icontains="producto")

In [ ]:
>> my_products = ProductModel.objects.get(id=12)
>> my_products.description
'Es una prueba'
>> my_products.price
199.99
>> my_products.price = 249.99
>>> my_products.save()
>> ProductModel.objects.get(id=11).delete()
(1, {'ecommerce.ProductModel': 1})

## Validacion de campos en los modelos

In [ ]:
ecommerce/validators.py

In [ ]:
from django.core.exceptions import ValidationError

BLOCKED_WORDS = [
    "barato",
    "malo"
]

def validate_blocked_words(value):
    init_string = f"{value}".lower()
    unique_words = set(init_string.split())
    blocked_words = set(BLOCKED_WORDS)
    invalid_words = (unique_words & blocked_words)
    has_error = len(invalid_words) > 0
    if has_error:
        errors = []
        for invalid_word in invalid_words:
            msg = f"{invalid_word} es una palabra no permitida"
            errors.append(msg)
        raise ValidationError(errors)
    return value

In [ ]:
ecommerce/models.py

In [ ]:
from django.db import models
from .validators import validate_blocked_words

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

## ¿Como agregar opciones a campos de modelos?

In [ ]:
from django.db import models
from .validators import validate_blocked_words

# (VALOR_EN_DB, VALOR_PARA_USUARIO)
PUBLISH_STATE_CHOICES = [
    ("BR", "BORRADOR"),
    ("PU", "PUBLICADO"),
    ("PR", "PRIVADO")
]

# Create your models here.
class ProductModel(models.Model):
    state = models.CharField(max_length=2, choices=PUBLISH_STATE_CHOICES, default="BR")
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

    def is_published(self):
        return self.state == "PU"

In [ ]:
python manage.py makemigrations
python manage.py migrate

## ¿Como agregar opciones avanzadas a campos de modelos?

In [ ]:
from django.db import models
from .validators import validate_blocked_words

# Create your models here.
class ProductModel(models.Model):
    class ProductStateOptions(models.TextChoices):
        PUBLISHED = "PU" , "PUBLICADO"
        RAFT = "BR" , "BORRADOR"
        PRIVATE = "PR", "PRIVADO"
    state = models.CharField(max_length=2, choices=PUBLISH_STATE_CHOICES, default="BR")
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

    def is_published(self):
        return self.state == self.ProductStateOptions.PUBLISHED

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Modelo abstarcto com base

In [ ]:
python manage.py startapp base

In [ ]:
base/models.py

In [ ]:
from django.db import models
from django.utils import timezone


class BasePublishModel(models.Model):
    class PublishStateOptions(models.TextChoices):
        PUBLISHED = "PU" , "PUBLICADO"
        RAFT = "BR" , "BORRADOR"
        PRIVATE = "PR", "PRIVADO"
    state = models.CharField(max_length=2, choices=PublishStateOptions.choices, default="BR")
    timestamp = models.DateField(auto_now=True)
    updated = models.DateField(auto_now=True)
    publish_timestamp = models.DateField(auto_now=False, null=True)

    class Meta:
        abstract = True
        ordering = ["-updated", "-timestamp"]

    def save(self, *args, **kwargs):
        if self.state_is_published and self.publish_timestamp is None:
            self.publish_timestamp = timezone.now()
        else:
            self.publish_timestamp = None
        super().save(*args, **kwargs)

    @property
    def state_is_published(self):
        return self.state == self.PublishStateOptions.PUBLISHED

    def is_published(self):
        publish_timestamp = self.publish_timestamp
        return self.state_is_published and publish_timestamp < timezone.now()

In [ ]:
ecommerce/models.py

In [ ]:
from django.db import models
from base.models import BasePublishModel
from .validators import validate_blocked_words

class ProductModel(BasePublishModel):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

In [ ]:
config/settings.py

In [ ]:
...

# Application definitions
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig",#<-----
    "base.apps.BaseConfig",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

...

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Creacion a granel

In [ ]:
products_data = []
for i in range(1, 100):
    new_data = {"title":"Producto {}".format(i) , "price": i*100+99.99}
    products_data.append(new_data)

In [ ]:
from ecommerce.models import ProductModel
new_objects = []
for product_data in products_data:
    print(product_data)
    new_objects.append(ProductModel(**product_data))

In [ ]:
ProductModel.objects.bulk_create(new_objects, ignore_conflicts=True)

## SlugField y Señales en modelos

In [ ]:
slug: se usa para los URL
title: escritorio con altura ajustable
slug: /escritorio-con-altura-ajustable
url: www.mywebsite.com/escritorio-con-altura-ajustable
url: www.mywebsite.com/1

Signals / Señales
- pre_save
- post_save
- pre_delete
- post_delete
- pre_init
- pre_migrate
- post_migrate

In [ ]:
from django.db.models import signals
dir(signals)

ecommerce/models.py

In [ ]:
from django.db import models
from base.models import BasePublishModel
from .validators import validate_blocked_words
from django.db.models.signals import pre_save
from django.utils.text import slugify

class ProductModel(BasePublishModel):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)
    slug = models.SlugField(null=True, blank=True, db_index=True)

    def get_absolute_url(self):
        return f"/product/{self.slug}"

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

def slugify_pre_save(sender, instance, *args, **kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        MyModel = instance.__class__
        qs = MyModel.objects.filter(slug__startswith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"

pre_save.connect(slugify_pre_save, sender=ProductModel)

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Fixtures para cargar datos

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json > ecommerce/fixtures/ecommerce/ProductModel.json

In [ ]:
python manage.py loaddata ecommerce/fixtures/ProductModel.json

## Llaves foraneas en modelos

In [ ]:
ecommerce/models.py

In [ ]:
from django.db import models
from base.models import BasePublishModel
from .validators import validate_blocked_words
from django.db.models.signals import pre_save
from django.utils.text import slugify
from django.conf import settings#<-----

User = settings.AUTH_USER_MODEL#<-----

class ProductModel(BasePublishModel):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)
    slug = models.SlugField(null=True, blank=True, db_index=True)
    user = models.ForeignKey(User, null=True, on_delete=models.SET_NULL)#<-----

    def get_absolute_url(self):
        return f"/product/{self.slug}"

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

def slugify_pre_save(sender, instance, *args, **kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        MyModel = instance.__class__
        qs = MyModel.objects.filter(slug__startswith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"

pre_save.connect(slugify_pre_save, sender=ProductModel)

In [ ]:
python manage.py makemigrations
python manage.py migrate